# Automated stress testing for PowerBI embeded Report without filters and page navigation

In [1]:
import threading
import time
import requests
import msal
from flask import Flask, render_template_string, abort
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
#Here apply your data extraction logic Your data should be stored in REPORTS[] where it shoud have


# report_name: name_of_the_report
# report_id: id_of_the_report


In [3]:
#like this
#REPORTS = [
    #{'report_id': 'xxxxxxxxxxxxxxxxxxxx', 'dataset_id': 'xxxxxxxxxxxxxxxx'}
    #]

In [ ]:
print("REPORTS = [")
for report in REPORTS:
    print(f'    {{\'report_id\': \'{report["report_id"]}\', \'dataset_id\': \'{report["dataset_id"]}\'}}')
print("]")

In [5]:
# --- Flask app setup ---
app = Flask(__name__)

# --- CONFIGURATION ---
#These credentials can be found in azure Entra > app registration and there you can find Client ID and Client secret and if you dont have any registration then you need to make one 
CLIENT_ID = "Enter_Client_ID"
CLIENT_SECRET = "Enter_Client_Secret"
TENANT_ID = "Enter_Tenan_ID"
WORKSPACE_ID = "Enter_Workspace_ID"
USERNAME = "Enter_Username"
ROLES = ["Enter_your_role"] #If there is no role set up remove it from token generation request

AUTHORITY_URL = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://analysis.windows.net/powerbi/api/.default"]

In [6]:
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Power BI Embedded Report {{ report_id }}</title>
    <script src="https://cdn.jsdelivr.net/npm/powerbi-client@2.19.1/dist/powerbi.min.js"></script>
</head>
<body>
    <h2>Embedded Power BI Report {{ report_id }}</h2>
    <div id="reportContainer" style="height: 700px; width: 100%;"></div>

    <script>
        const models = window['powerbi-client'].models;
        const embedConfig = {
            type: 'report',
            id: '{{ report_id }}',
            embedUrl: '{{ embed_url }}',
            accessToken: '{{ embed_token }}',
            tokenType: models.TokenType.Embed,
            settings: {
                panes: {
                    filters: { visible: false },
                    pageNavigation: { visible: true }
                }
            }
        };
        const reportContainer = document.getElementById('reportContainer');
        powerbi.embed(reportContainer, embedConfig);
    </script>
</body>
</html>
"""


In [7]:

def run_flask():
    app.run(debug=False, port=5000)

# Function to get access token from Power BI
def get_access_token():
    app_conf = msal.ConfidentialClientApplication(
        CLIENT_ID,
        authority=AUTHORITY_URL,
        client_credential=CLIENT_SECRET
    )
    token_result = app_conf.acquire_token_for_client(scopes=SCOPE)
    if "access_token" not in token_result:
        return None, token_result
    return token_result["access_token"], None

In [ ]:


# Flask route for index (listing available reports)
@app.route("/")
def index():
    links = [f'<li><a href="/report/{i}">Report {i+1}</a></li>' for i in range(len(REPORTS))]
    return f"<h1>Available Reports</h1><ul>{''.join(links)}</ul>"

# Flask route for embedding a report
@app.route("/report/<int:report_index>")
def embed_report(report_index):
    try:
        if report_index < 0 or report_index >= len(REPORTS):
            abort(404, description="Report not found")

        report_info = REPORTS[report_index]
        report_id = report_info["report_id"]
        dataset_id = report_info.get("dataset_id")

        if not dataset_id:
            # If dataset_id is missing, return a user-friendly error message
            return f"Error: No dataset found for report {report_id}."

        access_token, error = get_access_token()
        if error:
            return f"Error acquiring access token: {error}"

        headers = {"Authorization": f"Bearer {access_token}"}

        report_url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/reports/{report_id}"
        report_response = requests.get(report_url, headers=headers)

        if report_response.status_code != 200:
            print(f"Failed to get report. Status code: {report_response.status_code}")
            print(f"Error response: {report_response.text}")
            return f"Failed to retrieve report: {report_response.text}"

        embed_url = report_response.json().get("embedUrl")
        
        if not embed_url:
            print(f"Error: embed_url is missing for report {report_id}")
            return "Error: embed URL not found for the report."

        #please refer to token body documentation on powerbi website if the returns any code other than 200
        token_body = {
            "reports": [
             {"id": report_id, "groupId": WORKSPACE_ID}],
            "datasets": [{"id": dataset_id, "groupId": WORKSPACE_ID,"xmlaPermissions": "ReadOnly"}],
            "identities": [{
                "username": USERNAME,
                "roles": ROLES,
                "datasets": [dataset_id]
            }],
            "accessLevel": "View"
        }

        token_response = requests.post(
            "https://api.powerbi.com/v1.0/myorg/GenerateToken",
            headers={**headers, "Content-Type": "application/json"},
            json=token_body
        )

        if token_response.status_code != 200:
            print(f"Failed to generate embed token. Status code: {token_response.status_code}")
            print(f"Error response: {token_response.text}")
            return f"Failed to generate embed token: {token_response.text}"

        embed_token = token_response.json().get("token")

        if not embed_token:
            print(f"Error: embed_token is missing for report {report_id}")
            return "Error: embed token not found."

        return render_template_string(
            HTML_TEMPLATE,
            report_id=report_id,
            embed_url=embed_url,
            embed_token=embed_token
        )
    
    except Exception as e:
        # Catch all unexpected errors and return a generic error message
        print(f"Error: {str(e)}")
        return f"An unexpected error occurred: {str(e)}"

# --- Selenium setup and run ---
def run_selenium(num_reports):
    #customize these fileds
    brave_path = r"C:\Users\<YourUsername>\AppData\Local\BraveSoftware\Brave-Browser\Application" #enter your browser.exe path here
    
    chromedriver_path = r"C:\chromedriver\chromedriver-win64\chromedriver.exe" # enter your chrome driver path here
    
    options = Options()
    options.binary_location = brave_path
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-web-security")
    
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    
    print("Opening first report page...")
    driver.get(f"http://localhost:5007/report/0")
    time.sleep(5)  # Allow the first page to load

    # Open the remaining reports in new tabs
    for i in range(1, num_reports):
        print(f"Opening report {i} in a new tab...")
        driver.execute_script(f"window.open('http://localhost:5000/report/{i}', '_blank');")
        time.sleep(10)  # Allow the new tab to load

    # Keep browser open for a while (adjust as needed)
    #time.sleep(60)
    #driver.quit()

# Function to check if the server is up
def check_flask_server():
    url = "http://localhost:5000/"
    while True:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                print("Flask server is up and running!")
                return True
        except requests.exceptions.RequestException:
            pass  # Ignore errors and keep trying
        time.sleep(5)  # Wait before retrying

# Main execution block
if __name__ == "__main__":
    # Start Flask app in a thread
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    
    print("Waiting for Flask server to start...")
    
    # Wait until Flask server is ready to respond
    check_flask_server()

    # Open all report pages in browser tabs
    run_selenium(len(REPORTS))
